# Hybrid Recommender System

---------

## İş Problemi

**Senaryo**

ID'si verilen kullanıcı için item-based ve user-based recommender
yöntemlerini kullanarak 10 film önerisi yapınız.

## Veri Seti Hikayesi

*Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan
derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016
tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar
rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.*

In [ ]:
### Movie.csv Değişkenler

##### movieId : Eşsiz film numarası.
##### title : Film adı
##### genres : Tür

############################

### Rating.csv Değişkenler

##### userid : Eşsiz kullanıcı numarası. (UniqueID)
##### movieId : Eşsiz film numarası. (UniqueID)
##### rating : Kullanıcı tarafından filme verilen puan
##### timestamp : Değerlendirme tarihi 

## To do List :

### User Based Recommendation


**GÖREV 1**: Veri Hazırlama

**GÖREV 2**: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

**GÖREV 3**: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi

**GÖREV 4**: Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

**GÖREV 5**: Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması



### Item Based Recommendation

**GÖREV 1**: Kullanıcının izlediği en son ve en yüksek puan verdiği filme göre item-based öneri yapınız.



-------

### User Based Recommendation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Pandas kütüphanesini import edelim;
import pandas as pd
import numpy as np

# pandas görüntü ayarlarını yapalım;

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

# veri setlerini dahil edelim;

movie = pd.read_csv(r"/content/drive/MyDrive/recomReCap/dataset/movie_lens_dataset/movie.csv")
rating = pd.read_csv(r"/content/drive/MyDrive/recomReCap/dataset/movie_lens_dataset/rating.csv")

# veri setlerini birleştirelim;

df_ = movie.merge(rating,how="left", on = "movieId")
df = df_.copy()

df.head()


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


### Verileri Hazırlama

In [ ]:
# toplam oy sayısı 10000 altında olan filmleri veri setinden çıkaralım;

print(df.shape)

recom_count = pd.DataFrame(df["title"].value_counts())
rare_list = recom_count[recom_count["title"] < 10000].index.to_list()
df = df[~df["title"].isin(rare_list)]

print(df.shape)


(20000797, 6)
(9050403, 6)


In [ ]:
# pivot tablosunu oluşuturalım; 

pvt = df.pivot_table(index = ["userId"], columns = ["title"] , values = "rating")
pvt.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),...,Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.5,NaN,NaN,4.0,NaN,4.0
2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,5.0,4.0,4.0,5.0,NaN,NaN,NaN,5.0
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Süreci fonksiyonlaştıralım; 

def prep_data():
  movie = pd.read_csv(r"/content/drive/MyDrive/recomReCap/dataset/movie_lens_dataset/movie.csv")
  rating = pd.read_csv(r"/content/drive/MyDrive/recomReCap/dataset/movie_lens_dataset/rating.csv")
  df_ = movie.merge(rating,how="left", on = "movieId")

  recom_count = pd.DataFrame(df["title"].value_counts())
  rare_list = recom_count[recom_count["title"] > 30000].index.to_list()
  df = df[~df["title"].isin(rare_list)]

  pvt = df.pivot_table(index = ["userId"], columns = ["title"] , values = "rating")

  return pvt


### Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

In [ ]:
# rastgele bir kullanıcı belirleyelim; 

random_user = pd.Series(pvt.index).sample(1).values[0]

# kullanıcıya ait dataframe oluşturalım;

user_df = pvt[pvt.index == random_user]
user_df


title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),...,Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,
16060.0,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [ ]:
# kullanıcının oy kullandığı filmleri listeleyelim;

movies_watched = user_df.columns[user_df.notna().any()]

len(movies_watched)



144

### Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi

In [ ]:
# sadece kullanıcının izlediği filmleri baz alan bir df oluşturalım;

movies_watched_df = pvt[movies_watched]

# her bir kullanınıcının kaç tanesini izlediğini gösteren bir df oluşturalım;
user_movie_count = pd.DataFrame(movies_watched_df.T.notna().sum(), columns = ["count"])

user_movie_count

,count
userId,
1.0,43
2.0,11
3.0,39
4.0,9
5.0,23
...,...
138489.0,8
138490.0,5
138491.0,2


In [ ]:
# kullanıcının oyladığı filmleri %90'ını izleyen kullanıcıları seçelim;

users_same_movies = user_movie_count[user_movie_count["count"] > len(movies_watched)*.90].index.to_list()


### Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

In [ ]:
# sadece seçilen kullanıcıları içerecek şekilde pivot tablosunu filtreleyelim;

movies_watched_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]

In [ ]:
# kullanıcıların birbirleriyle olan ilişkisini oluşturan corr_df oluşturalım;

corr = movies_watched_df.T.corr().unstack().drop_duplicates().sort_values()

corr_df = pd.DataFrame(corr, columns = ["corr"])

corr_df.index.names = ["userId_1", "userId_2"]

corr_df.reset_index(inplace=True) 

corr_df

,userId_1,userId_2,corr
0,52009.0,107640.0,-0.314578
1,40285.0,107640.0,-0.261622
2,586.0,129420.0,-0.234204
3,60159.0,118955.0,-0.231467
4,87383.0,107640.0,-0.229285


In [ ]:
# seçilen kullanıcı ile 0.50 üstü korelasyon  kullanıcıları bulalım;

top_users = corr_df[(corr_df["userId_1"] == random_user) & (corr_df["corr"]> .50)][["userId_2","corr"]]

top_users.rename(columns = {"userId_2" : "userId"},inplace = True)

top_users = top_users.sort_values(by="corr", ascending=False).reset_index(drop = True)

top_users.head()

,userId,corr
0,93234.0,0.615564
1,107715.0,0.609207
2,137277.0,0.605649
3,93359.0,0.596088
4,82788.0,0.584582


In [ ]:
# top_users veri setini rating veri seti ile merge edelim;

top_users = top_users.merge(rating[["userId","movieId","rating"]],how="inner" , on="userId")

top_users

,userId,corr,movieId,rating
0,93234.0,0.615564,1,3.5
1,93234.0,0.615564,2,2.0
2,93234.0,0.615564,5,2.0
3,93234.0,0.615564,6,4.0
4,93234.0,0.615564,7,3.0
...,...,...,...,...
38299,34700.0,0.504993,65126,4.0
38300,34700.0,0.504993,65400,1.5
38301,34700.0,0.504993,65518,2.5
38302,34700.0,0.504993,65802,0.5


### Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

In [ ]:
# Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan weighted_rating adında yeni bir değişken oluşturalım;

top_users["weighted_average_rating"] = top_users["corr"] * top_users["rating"]

top_users.head()

,userId,corr,movieId,rating,weighted_average_rating
0,93234.0,0.615564,1,3.5,2.154475
1,93234.0,0.615564,2,2.0,1.231129
2,93234.0,0.615564,5,2.0,1.231129
3,93234.0,0.615564,6,4.0,2.462257
4,93234.0,0.615564,7,3.0,1.846693


In [ ]:
# Film id’si ve her bir filme ait tüm kullanıcıların weighted rating’lerinin ortalama değerini içeren dataframe oluşturalım;

recommendation_df = top_users.groupby(["movieId"])["weighted_average_rating"].mean().reset_index()

# recommendation_df içerisinde weighted rating'i 2.5'ten büyük olan filmleri seçiniz ve weighted rating’e göre sıralayınız.

recommendation_df = recommendation_df[recommendation_df["weighted_average_rating"] > 2].sort_values("weighted_average_rating" , ascending=False)

In [ ]:
# movie veri setinden film isimlerini getiriniz ve tavsiye edilecek ilk 5 filmi seçiniz.

recom = recommendation_df.merge(movie, how = "inner", on = "movieId")

# ilk 5 filmin ismini getirelim;
recom.head()["title"]

0                                        Priest (1994)
1                        Sexmission (Seksmisja) (1984)
2                              Teddy Bear (Mis) (1981)
3    Man with the Movie Camera, The (Chelovek s kin...
4                      Head-On (Gegen die Wand) (2004)
Name: title, dtype: object

### Item Based Recommendation

In [ ]:
# Seçili kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.

film_liste = []
for var in user_df:
  if user_df[var].values[0] == 5:
    film_liste.append(var)

film_liste

['Dark Knight, The (2008)',
 'Dogma (1999)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
 'Fight Club (1999)',
 'Kill Bill: Vol. 1 (2003)',
 'Kill Bill: Vol. 2 (2004)',
 'Lock, Stock & Two Smoking Barrels (1998)',
 'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
 'Monty Python and the Holy Grail (1975)',
 'Princess Bride, The (1987)',
 'Pulp Fiction (1994)',
 'Snatch (2000)']

In [ ]:
# Seçili kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.

select_df = df[(df["title"].isin(film_liste)) & (df["userId"] == random_user) ]
select_df.head()

,movieId,title,genres,userId,rating,timestamp
1915269,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,16060.0,5.0,2012-07-05 15:38:28
1947283,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,16060.0,5.0,2012-07-05 15:30:41
4576404,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,16060.0,5.0,2012-07-05 15:28:26
5961303,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,16060.0,5.0,2012-07-05 15:29:15
6180232,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,16060.0,5.0,2012-07-05 15:29:28


In [134]:
# en son 5 puan verilen filmi bulalım;

select_df["timestamp"] = pd.to_datetime(select_df["timestamp"])

last_movies = select_df[select_df["timestamp"]== select_df["timestamp"].max()]["title"].values[0]

last_movies

<ipython-input-134-401c4eced227>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_df["timestamp"] = pd.to_datetime(select_df["timestamp"])


'Kill Bill: Vol. 2 (2004)'

In [138]:
# en son 5 puan verilen filmin diğer filmler ile korelasyonunu hesaplayalım;

select_movies = pvt[last_movies]

item_corr = pvt.corrwith(select_movies).sort_values(ascending=False)

item_corr

title
Kill Bill: Vol. 2 (2004)     1.000000
Kill Bill: Vol. 1 (2003)     0.829857
Pulp Fiction (1994)          0.426959
Sin City (2005)              0.371244
Jackie Brown (1997)          0.325636
                               ...   
Casper (1995)                0.040587
Crocodile Dundee (1986)      0.034471
Executive Decision (1996)    0.028963
Dirty Dancing (1987)         0.028493
Santa Clause, The (1994)     0.007450
Length: 462, dtype: float64

In [149]:
# 5 kendisinden hariç 5 filmi tavsiye olarak oluşturalım;

recom_list = item_corr.iloc[1:6].index

for index,i in enumerate(recom_list):
  print(f"{index+1}. Önerilen Film: {i}")

1. Önerilen Film: Kill Bill: Vol. 1 (2003)
2. Önerilen Film: Pulp Fiction (1994)
3. Önerilen Film: Sin City (2005)
4. Önerilen Film: Jackie Brown (1997)
5. Önerilen Film: Reservoir Dogs (1992)
